In [248]:
import requests
import pandas as pd
import numpy as np
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize
import string
from sklearn.feature_extraction.text import CountVectorizer
from nltk.stem import WordNetLemmatizer
from sklearn.decomposition import LatentDirichletAllocation

In [249]:
df=pd.read_csv("/Users/fenice/code/QZKZ3/ht_project/raw_data/df_final_with_url_and_column_themes.csv")

In [250]:
text=df["text"]

In [251]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [252]:
word_to_delete=["know","really","would","day","said","two","one","thing","time","go","many","also","going","told","back","came","took","every","narrative","like","never","want","month","could","ing","didnt","made","could","think","get","come","went"] 


In [253]:
def remove_punct_and_word(item):
    for i in string.punctuation:
        item = item.replace(i, '')  
    return item.lower()
    
        
text=text.apply(remove_punct) 


In [254]:
def remove_manually_word(item):    
    for x in word_to_delete:
            item = item.replace(x, '')
    return item

text=text.apply(remove_manually_word) 


In [255]:
stop_words = set(stopwords.words('english')) 
def tokenize(item):
    word_tokens = word_tokenize(item)
    text = [w for w in word_tokens if not w in stop_words] 
    #lemmatize -> get root each tokenized word --> list
    
    #transform a list to string by putting space
    # " ".join(list)
    return text
text = text.apply(token)
text

0       [name, vi, 28, years, old, arrived, american, ...
1       [thank, opportunity, testify, problem, slavery...
2       [sister, liv, village, happy, poor, parents, s...
3       [grew, village, 500, 700, people, people, fami...
4       [liv, village, karko, nuba, mountains, parents...
                              ...                        
1353    [name, suleiman, ali, ’, sudanese, tell, trip,...
1354    [nigeria, difficult, make, liv, even, educatio...
1355    [name, seif, eldein, ’, 24, years, old, sudan,...
1356    [begg, sleep, street, behind, house, ask, pass...
1357    [lived, mother, father, poor, opportunity, tra...
Name: text, Length: 1358, dtype: object

In [256]:
lemmatizer = WordNetLemmatizer()

def lemmat(item):
    lemmatized = [lemmatizer.lemmatize(word) for word in item]
    return lemmatized

text=text.apply(lemmat)

In [257]:

def trans_string(item):
    return  " ".join(item)
text=text.apply(trans_string)


In [258]:
vectorizer = CountVectorizer()

X = vectorizer.fit_transform(text)


In [259]:
new_df=pd.DataFrame(X.toarray(),columns = vectorizer.get_feature_names())

In [261]:
lda_model = LatentDirichletAllocation(n_components=5).fit(new_df)

def print_topics(model, vectorizer):
    for idx, topic in enumerate(model.components_):
        print("Topic %d:" % (idx))
        print([(vectorizer.get_feature_names()[i], topic[i])
                        for i in topic.argsort()[:-10 - 1:-1]])
        

print_topics(lda_model, vectorizer)

Topic 0:
[('woman', 392.6638742834848), ('life', 213.9763204139304), ('men', 197.82008642399822), ('family', 185.63157704342092), ('school', 178.85018732764703), ('man', 172.66848676597752), ('year', 169.39917905191962), ('north', 165.85546768140978), ('child', 156.1796957930959), ('house', 153.6743865768796)]
Topic 1:
[('work', 2379.843536091777), ('pay', 779.2533992433046), ('house', 762.7365954873146), ('family', 747.9420049050984), ('people', 724.8325591444047), ('year', 702.1405021347101), ('home', 683.1466310579227), ('asked', 610.6085758666723), ('police', 608.938856592663), ('even', 596.7613875335479)]
Topic 2:
[('people', 987.955737657825), ('life', 785.5635697406616), ('girl', 665.210196164069), ('child', 600.3227848146112), ('help', 574.1436360244486), ('say', 544.7519316152744), ('even', 525.2832963964667), ('traffick', 523.4563351878503), ('see', 474.1999210923063), ('feel', 474.11857208811085)]
Topic 3:
[('mother', 752.5018308596233), ('child', 579.5567853953426), ('husba

In [286]:
df=pd.read_csv("/Users/fenice/code/QZKZ3/ht_project/raw_data/dfht_25-11_13-02.csv")

In [287]:
df.isna().sum()[df.isna().sum()>10]

name                   26
year                   26
departure             142
arrival                95
date_slavery         1204
arrival_1              95
arrival_2            1277
arrival_3            1333
theme_2               920
date_slavery_from    1205
date_slavery_to      1205
dtype: int64

In [293]:
df.drop(columns=["date_slavery","theme","arrival"],inplace=True)

In [294]:
df

,text,name,year,departure,latitude,longitude,gender,accuracy,meansOfControlDebtBondage,meansOfControlExcessiveWorkingHours,...,url,arrival_1,arrival_2,arrival_3,theme_1,theme_2,date_slavery_from,date_slavery_to,bought_sold_for,any_money_mention
0,My name is Vi. And I am 28 years old. I arrive...,Vi,2001.0,Vietnam,-14.354000,-170.773600,female,67.0,"['debt', 'debts']",[],...,http://antislavery.ac.uk/items/show/7,American Samoa,NaN,NaN,Forced labour,NaN,NaN,NaN,[],"[('408', '$'), ('4000', '$'), ('2000', '$'), (..."
1,Thank you for the opportunity to testify befor...,Beatrice,2005.0,Sri Lank,33.900000,35.500000,female,98.0,[],[],...,http://antislavery.ac.uk/items/show/8,Lebanon,NaN,NaN,Domestic slavery,NaN,NaN,NaN,[],[]
2,My sister and I were living in a village. We w...,Kavita,2004.0,Sri Lank,24.071318,79.652396,female,97.0,[],[],...,http://antislavery.ac.uk/items/show/102,India,NaN,NaN,Domestic slavery,NaN,2002.0,2004.0,[],[]
3,I grew up in a village of between 500 and 700 ...,Maria,2005.0,Mexico,40.412044,-96.870925,female,100.0,[],[],...,http://antislavery.ac.uk/items/show/103,United States,NaN,NaN,Domestic slavery,NaN,1976.0,1981.0,[],"[('200', '$')]"
4,"I was living in a village, Karko, in the Nuba ...",Mende,2003.0,Sudan,16.865226,32.271106,male,90.0,[],[],...,http://antislavery.ac.uk/items/show/104,Sudan,United Kingdom,NaN,War slavery,NaN,1994.0,2000.0,[],[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1353,"My name is Suleiman Ali, I’m Sudanese. I want ...",Suleiman,2020,Egypt,26.335100,17.228331,male,100.0,[],[],...,http://antislavery.ac.uk/items/show/2949,Libya,NaN,NaN,Trafficking,NaN,NaN,NaN,[],[]
1354,"In Nigeria, it was difficult to make a living....",Shola,2020,Nigeria,26.335100,17.228331,male,80.0,[],[],...,http://antislavery.ac.uk/items/show/2950,Libya,NaN,NaN,too much themes,NaN,NaN,NaN,[],[]
1355,"My name is Seif Eldein. I’m 24 years old, from...",Seif,2020,Chad,26.335100,17.228331,male,100.0,[],[],...,http://antislavery.ac.uk/items/show/2951,Libya,NaN,NaN,Trafficking,Forced labour,NaN,NaN,[],[]
1356,I was begging and sleeping in the street behin...,Rita,2020,Nigeria,43.769562,11.255814,female,100.0,[],[],...,http://antislavery.ac.uk/items/show/2952,Italy,Libya,NaN,Sexual exploitation,Trafficking,NaN,NaN,[],[]


In [298]:
df.to_csv("final_df_26_11.csv", index=False)

In [300]:
df["arrival_1"][df["departure"]=="Mexico"]

Series([], Name: arrival_1, dtype: object)

In [316]:
def strip(item):
    try: 
        item=item.strip()
        return item
    except:
        return item
df=df.applymap(strip)

In [317]:
df.loc[0,"departure"]

'Vietnam'

In [318]:
df.to_csv("final_df_26_11.csv", index=False)

In [321]:
pd.read_csv("/Users/fenice/code/QZKZ3/ht_project/raw_data/final_df_26_11.csv").loc[0,"departure"]

'Vietnam'

In [329]:
df[(df["departure"] == "Sri Lank") &
                                  (df["gender"] == "female")]

,text,name,year,departure,latitude,longitude,gender,accuracy,meansOfControlDebtBondage,meansOfControlExcessiveWorkingHours,...,url,arrival_1,arrival_2,arrival_3,theme_1,theme_2,date_slavery_from,date_slavery_to,bought_sold_for,any_money_mention
1,Thank you for the opportunity to testify befor...,Beatrice,2005.0,Sri Lank,33.900000,35.500000,female,98.0,[],[],...,http://antislavery.ac.uk/items/show/8,Lebanon,NaN,NaN,Domestic slavery,NaN,NaN,NaN,[],[]
2,My sister and I were living in a village. We w...,Kavita,2004.0,Sri Lank,24.071318,79.652396,female,97.0,[],[],...,http://antislavery.ac.uk/items/show/102,India,NaN,NaN,Domestic slavery,NaN,2002.0,2004.0,[],[]


In [335]:
df

,text,name,year,departure,latitude,longitude,gender,accuracy,meansOfControlDebtBondage,meansOfControlExcessiveWorkingHours,...,url,arrival_1,arrival_2,arrival_3,theme_1,theme_2,date_slavery_from,date_slavery_to,bought_sold_for,any_money_mention
0,My name is Vi. And I am 28 years old. I arrive...,Vi,2001.0,Vietnam,-14.354000,-170.773600,female,67.0,"['debt', 'debts']",[],...,http://antislavery.ac.uk/items/show/7,American Samoa,NaN,NaN,Forced labour,NaN,NaN,NaN,[],"[('408', '$'), ('4000', '$'), ('2000', '$'), (..."
1,Thank you for the opportunity to testify befor...,Beatrice,2005.0,Sri Lank,33.900000,35.500000,female,98.0,[],[],...,http://antislavery.ac.uk/items/show/8,Lebanon,NaN,NaN,Domestic slavery,NaN,NaN,NaN,[],[]
2,My sister and I were living in a village. We w...,Kavita,2004.0,Sri Lank,24.071318,79.652396,female,97.0,[],[],...,http://antislavery.ac.uk/items/show/102,India,NaN,NaN,Domestic slavery,NaN,2002.0,2004.0,[],[]
3,I grew up in a village of between 500 and 700 ...,Maria,2005.0,Mexico,40.412044,-96.870925,female,100.0,[],[],...,http://antislavery.ac.uk/items/show/103,United States,NaN,NaN,Domestic slavery,NaN,1976.0,1981.0,[],"[('200', '$')]"
4,"I was living in a village, Karko, in the Nuba ...",Mende,2003.0,Sudan,16.865226,32.271106,male,90.0,[],[],...,http://antislavery.ac.uk/items/show/104,Sudan,United Kingdom,NaN,War slavery,NaN,1994.0,2000.0,[],[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1353,"My name is Suleiman Ali, I’m Sudanese. I want ...",Suleiman,2020,Egypt,26.335100,17.228331,male,100.0,[],[],...,http://antislavery.ac.uk/items/show/2949,Libya,NaN,NaN,Trafficking,NaN,NaN,NaN,[],[]
1354,"In Nigeria, it was difficult to make a living....",Shola,2020,Nigeria,26.335100,17.228331,male,80.0,[],[],...,http://antislavery.ac.uk/items/show/2950,Libya,NaN,NaN,too much themes,NaN,NaN,NaN,[],[]
1355,"My name is Seif Eldein. I’m 24 years old, from...",Seif,2020,Chad,26.335100,17.228331,male,100.0,[],[],...,http://antislavery.ac.uk/items/show/2951,Libya,NaN,NaN,Trafficking,Forced labour,NaN,NaN,[],[]
1356,I was begging and sleeping in the street behin...,Rita,2020,Nigeria,43.769562,11.255814,female,100.0,[],[],...,http://antislavery.ac.uk/items/show/2952,Italy,Libya,NaN,Sexual exploitation,Trafficking,NaN,NaN,[],[]
